In [ ]:
# fundamental packages
import streamlit as st
import pandas as pd
import numpy as np

# ds packages
from prophet import Prophet

# viz packages
import matplotlib.pyplot as plt
import seaborn as sns

# MLOps
from snowflake.ml.registry import Registry #snowflake-ml-python
from snowflake.ml._internal.utils import identifier
from snowflake.ml.model import custom_model
import cloudpickle

# active session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
SELECT *
FROM DEMO_ENERGY.BRONZE.KAGGLE_OILWELL;

# Check & Modify Our Data
![Alt text](https://miro.medium.com/v2/resize:fit:1400/0*5UXauCfV0YoVK8YG)

In [ ]:
df = GetData_SQL.to_pandas()
df.head()

In [ ]:
df.describe()

In [ ]:
print(df.dtypes)

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y-%m-%d')

print(df.dtypes)
df.head()

In [ ]:
df.columns = ['Date', 'Oil Vol.', 'Liquid Vol.', 'Gas Vol.',
              'Water Vol.', 'Water Cut%', 'Working Hours', 'Dynamic Level', 'Reservoir Pressure (atm)']

df.info()

# Visual Analysis
![Alt Tex](https://i.imgur.com/Bx4AkTJ.png)

In [ ]:
fig = plt.figure(figsize=(20,20))
spec = fig.add_gridspec(4,2)
ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[1,0])
ax4 = fig.add_subplot(spec[1,1])
ax5 = fig.add_subplot(spec[2,0])
ax6 = fig.add_subplot(spec[2,1])
ax7 = fig.add_subplot(spec[3,0])
ax8 = fig.add_subplot(spec[3,1])

axs = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8]

# Adding color preference
bg_color = '#FFF7E9'
for selection in axs:
    selection.set_facecolor(bg_color)

i = 0

for col in df.columns[1:9]:
    axs[i].plot(df["Date"],df[col],lw=0.34,linestyle="dotted",c="#D09CFA")
    axs[i].plot(df["Date"],df[col].rolling(window=30).mean(),c="#FF731D")
    axs[i].set_title(f"{col} vs Time",fontsize=16)
    i+=1
    
plt.show()

# About Oil Volume vs Water Cut vs Rest
Upstream oil analysis is key to optimizing production by analyzing the time series relationship between **Oil Volume, Water Cut Percent, and Down Time**. Tracking these metrics helps identify trends and correlations that impact production efficiency. For example, increasing water cut could indicate equipment issues or well depletion, while downtime directly affects oil volume. By examining these variables together, operators can pinpoint inefficiencies, reduce unplanned downtime, and improve reservoir management.  
 * **Oil Volume** indicates the amount of crude oil produced over time, serving as a core measure of production performance.
  
 * **Water Cut Percent** refers to the proportion of water in the produced fluids, which can signal issues like reservoir depletion, equipment malfunctions, or the need for well interventions.
  
 * **Down Time** measures the periods when production is halted, often due to mechanical failures, maintenance, or operational inefficiencies.

In [ ]:
df["Down Time"] = 24 - df["Working Hours"]

fig = plt.figure(figsize=(20,12))
spec = fig.add_gridspec(1,1)
ax1 = fig.add_subplot(spec[0,0])

# Plotting Oil volume and Rest Time on the primary y-axis
ax1.plot(df["Date"], df["Oil Vol."].rolling(window=30).mean(), c="purple")
ax1.plot(df["Date"], df["Down Time"].rolling(window=30).mean(), c="#FF731D")
ax1.set_xlabel('Date')

# Adding legend for the primary y-axis
ax1.legend(["Oil volume (m3/day)", "Down Time"], bbox_to_anchor=(1,1))

ax1.set_ylabel("Time (hr) (OR) Volume(m3/day)")

# Creating secondary y-axis for Water cut (%)
ax2 = ax1.twinx()
ax2.plot(df["Date"], df['Water Cut%'].rolling(window=30).mean(), c="grey",linestyle="dotted")
ax2.set_ylabel('Water cut (%)')

# Adding legend for the secondary y-axis
ax2.legend(["Water cut (%)"], bbox_to_anchor=(0.965,0.93))

ax2.set_title("Oil Volume (liters/day) vs Water Cut (%) vs Down Time",fontsize="14")

vertical_line_date1 = '2015-06-01'  # Example date for the vertical line
ax2.axvline(pd.to_datetime(vertical_line_date1), color='black', linestyle='--', label='Event',lw=0.5)

vertical_line_date2 = '2016-06-01'  # Example date for the vertical line
ax2.axvline(pd.to_datetime(vertical_line_date2), color='black', linestyle='--', label='Event',lw=0.5)

ax2.patch.set_alpha(0)
ax2.grid(False)

plt.show()

# Forecast Oil Volume

Using the Facebook Prophet time series package, forecast oil volume.

#### About the Package:
Copy+Paste from github: https://facebook.github.io/prophet/  
  
*Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.*

*Prophet is open source software released by Facebook’s Core Data Science team. It is available for download on CRAN and PyPI.*

**Accurate and Fast**  
*Prophet is used in many applications across Facebook for producing reliable forecasts for planning and goal setting. We’ve found it to perform better than any other approach in the majority of cases. We fit models in Stan so that you get forecasts in just a few seconds.*

**Fully Automatic**  
*Get a reasonable forecast on messy data with no manual effort. Prophet is robust to outliers, missing data, and dramatic changes in your time series.*

**Tunable Forecasts**  
*The Prophet procedure includes many possibilities for users to tweak and adjust forecasts. You can use human-interpretable parameters to improve your forecast by adding your domain knowledge.*

#### Resources:
Link to Quick Start [https://facebook.github.io/prophet/docs/quick_start.html]  

![Alt Text](https://pwlobby.b-cdn.net/wp-content/uploads/2023/04/Ron-Burgandy-forecast-meme.jpg)

In [ ]:
# Packages for Modeling
from prophet import Prophet
from snowflake.ml.registry import Registry
from snowflake.ml.model import custom_model
from snowflake.ml.model import model_signature

In [ ]:
df_prophet = df.reset_index()[['Date', 'Oil Vol.']]
df_prophet = df_prophet.rename(columns={'Date': 'DATE', 'Oil Vol.': 'OIL_VOL'})

df_prophet

In [ ]:
# Save to Silver Schema for others to use
df_silver = session.create_dataframe(df_prophet)
df_silver.write.mode("overwrite").save_as_table("DEMO_ENERGY.SILVER.KAGGLE_OILVOL_TS_V3") # Update the version number as needed to demonstrate uploading to snow

In [ ]:
-- Demonstrate table saved to silver and we can now query using SQL
SELECT *
FROM DEMO_ENERGY.SILVER.KAGGLE_OILVOL_TS_V3 -- Update version number to mirror cell above.

In [ ]:
#Pull Data (as pandas dataframe) for modeling
#Need to rename.
#Prophet requires columns "ds" and "y" with the dates and values respectively.
df = QueryUsingSQL.to_pandas().rename(columns={'DATE': 'ds', 'OIL_VOL': 'y'})

df.info()

In [ ]:
#Initialize Prophet model
oil_forcast_model = Prophet()

oil_forcast_model.fit(df)

future = oil_forcast_model.make_future_dataframe(periods = 365)
forecast = oil_forcast_model.predict(future)

# View the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

In [ ]:
forecast

In [ ]:
fig, ax = plt.subplots()

ax.fill_between(
    x = forecast['ds'],
    y1 = forecast['yhat_lower'],
    y2 = forecast['yhat_upper'],
    alpha = 0.25,
    label = r'0.95 credible_interval'
)

sns.lineplot(x='ds', y='y', label='Oil Volume', data=df, ax=ax)
sns.lineplot(x='ds', y='yhat', label='Y-Hat', data=forecast, ax=ax)
ax.legend(loc='upper left')
ax.set(title='Oil Volume vs Time', ylabel='Liters per Day', xlabel='Time');

In [ ]:
# Save to Gold Schema for others to use
df_gold = session.create_dataframe(forecast)
df_gold.write.mode("overwrite").save_as_table("DEMO_ENERGY.GOLD.KAGGLE_OILVOL_FORECAST_V2")

In [ ]:
SELECT *
FROM DEMO_ENERGY.GOLD.KAGGLE_OILVOL_FORECAST_V2

# Put Model into Production

![Alt Text](https://miro.medium.com/v2/resize:fit:940/1*8d2JAxR_4IMXelZ8TsKiTA.png)

In [ ]:
CREATE OR REPLACE STAGE DEMO_ENERGY.GOLD.MODEL_STAGE;

In [ ]:
cloudpickle.dump(oil_forcast_model, open('oil_forcast_model.pkl', 'wb'))

## Uplad the model into stage
session.file.put("oil_forcast_model.pkl", "@DEMO_ENERGY.GOLD.MODEL_STAGE/", auto_compress=False)

In [ ]:
ls @DEMO_ENERGY.GOLD.MODEL_STAGE

In [ ]:
# Initialize ModelContext with keyword arguments
# my_model can be any supported model type
# my_file_path is a local pickle file path

mc = custom_model.ModelContext(
    artifacts={
        'config': 'oil_forcast_model.pkl'
    }
)


# Define a custom model class that utilizes the context
class MyProphetModel(custom_model.CustomModel):

    def __init__(self,context:custom_model.ModelContext) -> None:
        super().__init__(context)
        ## use 'file_path to load the piecked object
        with open(self.context['config'],'rb') as f:
            self.model =pickle.load(f)
    @custom_model.inference_api
    def predict(self,X:pd.DataFrame) -> pd.DataFrame:
        X_copy = X.copy()
        X_copy['ds']=pd.to_datetime(X_copy['ds'])# ensure correrct datetime
        forecast = self.model.predict(X_copy)
        res_df = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]
        return res_df

In [ ]:
df.head(5)

In [ ]:
def create_datedf(max_date, period=7): 
    # Create a new DataFrame starting from max_date + 1 day, adding 7 days to each subsequent row
    date_range = pd.date_range(start=max_date + pd.Timedelta(days=1), periods=period)
    new_df = pd.DataFrame({'ds': date_range})
    return new_df

max_date = df['ds'].max()

#max_date

df_1 = create_datedf(max_date, period=10)

## prediction on test data
forecast_model = MyProphetModel(mc)

output = forecast_model.predict(df_1)
output.head(5)

# Manage models using Model Registry
  
Use Snowpark model registry. This is the Snowflake native model versioning and deployment framework. This allows us to log models, tag parameters and metrics, track metadata, create versions, and ultimately execute batch inference tasks in a Snowflake warehouse or deploy to a Snowpark Container Service.

In [ ]:
reg = Registry(session,database_name = "DEMO_ENERGY",schema_name= "GOLD")

custom_mv = reg.log_model(
   forecast_model,
    model_name="Prophet_forcast_model",
    version_name="v3",
    conda_dependencies=["prophet"],
    sample_input_data= df_1,
    options={'relax_version': False},
    comment = 'My Prophet forcast experiment using the CustomModel API'
)

In [ ]:
reg.show_models()

reg = Registry(session=session, database_name='DEMO_ENERGY', schema_name='GOLD')

mv = reg.get_model('Prophet_forcast_model').version('v1')

pr = mv.run(df_1, function_name='PREDICT')

In [ ]:
df_1.head()

In [ ]:
pr